# Nguyễn Đồng Thanh - 23127538 - Lab2: Numpy for Data Science

# 1. Thông tin về dữ liệu

Để hiểu tập dữ liệu hơn, chúng ta cùng lên Kaggle và đọc một vài thông tin liên quan được trình bày trên đó. 

Một số thông tin em tìm được:

- **Nguồn dữ liệu:** [Kaggle - Credit Card Customers](https://www.kaggle.com/datasets/sakshigoyal7/credit-card-customers)
- **Tác giả:** Sakshi Goyal
- **Bài toán:** Dự đoán khách hàng nào sẽ rời bỏ dịch vụ ngân hàng (Churn Prediction) để ngân hàng có thể chủ động giữ chân họ.

**Ý nghĩa các cột dữ liệu quan trọng:**
Bộ dữ liệu này bao gồm thông tin về nhân khẩu học, mối quan hệ với ngân hàng và lịch sử giao dịch:

* **CLIENTNUM:** Mã định danh duy nhất của khách hàng.
* **Attrition_Flag:** Biến mục tiêu (Target Variable). Cho biết khách hàng còn sử dụng dịch vụ (`Existing Customer`) hay đã rời bỏ (`Attrited Customer`).
* **Customer_Age, Gender, Dependent_count:** Tuổi, Giới tính, Số người phụ thuộc.
* **Education_Level:** Trình độ học vấn (High School, Graduate, Uneducated, etc.).
* **Marital_Status:** Tình trạng hôn nhân.
* **Income_Category:** Nhóm thu nhập hàng năm.
* **Card_Category:** Loại thẻ (Blue, Silver, Gold, Platinum).
* **Months_on_book:** Thời gian khách hàng đã gắn bó với ngân hàng (tính bằng tháng).
* **Total_Relationship_Count:** Tổng số sản phẩm/dịch vụ khách hàng đang sử dụng.
* **Months_Inactive_12_mon:** Số tháng không hoạt động trong 12 tháng qua.
* **Contacts_Count_12_mon:** Số lần liên hệ với ngân hàng trong 12 tháng qua.
* **Credit_Limit:** Hạn mức tín dụng.
* **Total_Revolving_Bal:** Tổng số dư nợ quay vòng.
* **Avg_Open_To_Buy:** Hạn mức tín dụng còn khả dụng trung bình (Credit Limit - Revolving Balance).
* **Total_Trans_Amt:** Tổng số tiền giao dịch (trong 12 tháng qua).
* **Total_Trans_Ct:** Tổng số lần giao dịch (trong 12 tháng qua).
* **Avg_Utilization_Ratio:** Tỷ lệ sử dụng thẻ trung bình.

## Import thư viện

In [1]:
# Import câc thư viện cần thiết

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

# Cấu hình hiển thị
sns.set_theme(style="whitegrid")
warnings.filterwarnings('ignore')

## Khai báo hàm

In [2]:
def load_data_numpy(filepath: str) -> np.ndarray:
    """
    Đọc dữ liệu CSV bằng NumPy.
    Input:
        filepath (str): Đường dẫn tới file CSV.
    Output:
        np.ndarray: Structured array chứa toàn bộ dữ liệu kèm tên cột.
    """
    # Dùng genfromtxt để giữ nguyên tên cột và kiểu dữ liệu; dtype=None để NumPy tự suy đoán.
    if not os.path.exists(filepath):
            print(f"Lỗi: Không tìm thấy file tại đường dẫn: {filepath}")
            return None

    try:
        data = np.genfromtxt(filepath, delimiter=',', dtype=None, names=True, encoding='utf-8')
        print("Đã load dữ liệu thành công!")
        return data
    except Exception as e:
        print(f"Có lỗi xảy ra khi đọc file: {e}")
        return None


def get_dataset_overview(data):
    """
    Mô phỏng hàm .info() của Pandas bằng Numpy.
    In ra tên cột, kiểu dữ liệu và số lượng giá trị (count).
    
    Input: data (structured array)
    """
    print(f"{'Column Name':<30} {'Data Type':<15} {'Count':<10}")
    print("-" * 60)
    for name in data.dtype.names:
        # Lấy kiểu dữ liệu
        dtype = data.dtype[name]
        # Đếm số lượng phần tử (shape[0])
        count = data.shape[0]
        print(f"{name:<30} {str(dtype):<15} {count:<10}")

def describe_numeric_feature(data, column_name):
    """
    Mô phỏng hàm .describe() cho 1 cột số bằng Numpy.
    
    Input: 
        data: structured array
        column_name: tên cột cần thống kê
    Output: 
        Dictionary chứa các chỉ số thống kê
    """
    col_data = data[column_name]
    
    # Kiểm tra xem có phải cột số không
    if not np.issubdtype(col_data.dtype, np.number):
        return f"Cột {column_name} không phải dạng số."
    
    desc = {
        'Mean': np.mean(col_data),
        'Median': np.median(col_data),
        'Std': np.std(col_data),
        'Min': np.min(col_data),
        'Max': np.max(col_data),
        '25%': np.percentile(col_data, 25),
        '75%': np.percentile(col_data, 75)
    }
    return desc

def describe_categorical_feature(data, column_name):
    """
    Mô phỏng hàm .describe() cho 1 cột phân loại (categorical) bằng Numpy.
    
    Input: 
        data: structured array
        column_name: tên cột cần thống kê
    Output: 
        Dictionary chứa các chỉ số thống kê cho dữ liệu phân loại
    Nhiệm vụ:
        Đếm số giá trị unique, tìm giá trị xuất hiện nhiều nhất và tần số của nó
    """
    col_data = data[column_name]
    
    # Kiểm tra xem có phải cột phân loại (không phải số) không
    if np.issubdtype(col_data.dtype, np.number):
        return f"Cột {column_name} là dạng số, không phải categorical."
    
    # Tìm các giá trị unique và đếm số lần xuất hiện
    unique_values, counts = np.unique(col_data, return_counts=True)
    
    # Tìm giá trị xuất hiện nhiều nhất (mode)
    max_count_idx = np.argmax(counts)
    most_common_value = unique_values[max_count_idx]
    most_common_count = counts[max_count_idx]
    
    desc = {
        'Unique Count': len(unique_values),
        'Most Common Value': most_common_value,
        'Most Common Frequency': most_common_count,
        'Most Common Percentage': f"{(most_common_count / len(col_data)) * 100:.2f}%"
    }
    return desc


def describe_all(data, include=False):
    """
    Mô phỏng hàm .describe() của Pandas cho toàn bộ dataset bằng Numpy.
    
    Input:
        data: structured array
        include (bool): 
            - False: chỉ in thống kê các cột số
            - True: in thống kê cả cột số và cột phân loại
    Output:
        In ra thống kê mô tả của các cột
    Nhiệm vụ:
        Tự động phân loại và in thống kê cho từng loại cột
    """
    # Tách cột số và cột phân loại
    numeric_cols = [name for name in data.dtype.names if np.issubdtype(data.dtype[name], np.number)]
    categorical_cols = [name for name in data.dtype.names if not np.issubdtype(data.dtype[name], np.number)]
    
    # In thống kê các cột số
    print("="*80)
    print("THỐNG KÊ CÁC CỘT SỐ")
    print("="*80)
    
    for col in numeric_cols:
        print(f"\nCột: {col}")
        print("-" * 60)
        desc = describe_numeric_feature(data, col)
        
        for key, value in desc.items():
            print(f"  {key:<15}: {value:>12.2f}")
    
    # Nếu include=True thì in thêm thống kê các cột phân loại
    if include:
        print("\n\n")
        print("="*80)
        print("THỐNG KÊ CÁC CỘT PHÂN LOẠI")
        print("="*80)
        
        for col in categorical_cols:
            print(f"\nCột: {col}")
            print("-" * 60)
            desc = describe_categorical_feature(data, col)
            
            for key, value in desc.items():
                print(f"  {key:<25}: {value}")



def compute_correlation_matrix(data):
    """
    Tính ma trận tương quan Pearson cho các cột số bằng Numpy.
    Tự động mã hóa cột 'Attrition_Flag' (Existing -> 0, Attrited -> 1) để tính tương quan với Target.
    
    Input: data (structured array)
    Output: 
        corr_matrix: ma trận tương quan (numpy array)
        feature_names: danh sách tên các cột được dùng tính toán
    """
    # 1. Xác định các cột số
    numeric_cols = [name for name in data.dtype.names if np.issubdtype(data.dtype[name], np.number)]
    
    # Loại bỏ cột ID nếu có (thường không có ý nghĩa thống kê)
    if 'CLIENTNUM' in numeric_cols:
        numeric_cols.remove('CLIENTNUM')
        
    # 2. Xử lý đặc biệt cho Target 'Attrition_Flag' để đưa vào ma trận tương quan
    # Tạo mảng tạm chứa các giá trị số
    temp_list = []
    
    # Xử lý Target: Attrited Customer -> 1, Existing Customer -> 0
    target_col = np.where(data['Attrition_Flag'] == 'Attrited Customer', 1, 0)
    temp_list.append(target_col)
    feature_names = ['Attrition_Flag'] + numeric_cols # Đưa Target lên đầu
    
    # Lấy dữ liệu các cột số khác
    for col in numeric_cols:
        temp_list.append(data[col])
        
    # 3. Stack lại thành ma trận 2D (Features x Samples) -> Transpose thành (Samples x Features)
    # np.column_stack giúp ghép các mảng 1D thành các cột của ma trận 2D
    data_matrix = np.column_stack(temp_list)
    
    # 4. Tính ma trận tương quan bằng np.corrcoef
    # rowvar=False nghĩa là mỗi cột là một biến số
    corr_matrix = np.corrcoef(data_matrix, rowvar=False)
    
    return corr_matrix, feature_names


## Load dữ liệu

In [3]:
csv_path = "../data/raw/original_data.csv"

# Load dữ liệu
data_np = load_data_numpy(csv_path)

# Thông tin nhanh về dữ liệu
print("Kích thước dữ liệu (rows, cols):", data_np.shape)
print("Tên các cột:")
print(data_np.dtype.names)

print(f"\n3 dòng dữ liệu đầu tiên:")
print(data_np[:3])

Đã load dữ liệu thành công!
Kích thước dữ liệu (rows, cols): (10127,)
Tên các cột:
('CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender', 'Dependent_count', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category', 'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2')

3 dòng dữ liệu đầu tiên:
[(768805383, '"Existing Customer"', 45, '"M"', 3, '"High School"', '"Married"', '"$60K - $80K"', '"Blue"', 39, 5, 1, 3, 12691., 777, 11914., 1.335, 1144, 42, 1.625, 0.061, 9.3448e-05, 0.99991)
 (818770008, '"E

# 2. Khám phá dữ liệu

**Chiến lược phân tích**

1.  **Xem thống kê mô tả:** Tạo các hàm giống `.info()` hay `.describe()` của Pandas bằng numpy:  tính toán các chỉ số cơ bản (mean, median, min, max, std) cho các biến số và đếm tần suất (unique counts) cho các biến phân loại.
2.  **Quan sát từng cột có ý nghĩa cao:** Vì có tới 23 cột, quá nhiều để quan sát chi tiết tất cả các cột, em sẽ chọn ra những cột có ý nghĩa cao (dựa vào kinh nghiệm, ngữ nghĩa, thông tin đọc được trên Kaggel, ma trận tương quan). Sau đó tiến hành quan sát chi tiết các cột đó vẽ biểu đồ phân phối (histogram/bar chart) để xem dữ liệu có bị lệch hay có giá trị ngoại lai không.
3.  **Quan sát tương quan :** Xem xét mối quan hệ giữa các biến ý nghĩa (đặc biệt là với biến target `Attrition_Flag`), vẽ thêm các biểu đồ đường, chấm, cột để quan mối quan hệ, sự tương quan. 

## 2.1. Tổng quan dữ liệu

Sử dụng hàm `get_dataset_overview` để kiểm tra kiểu dữ liệu và số lượng mẫu.

In [4]:
# Xem thông tin tổng quan các cột
get_dataset_overview(data_np)

Column Name                    Data Type       Count     
------------------------------------------------------------
CLIENTNUM                      int64           10127     
Attrition_Flag                 <U19            10127     
Customer_Age                   int64           10127     
Gender                         <U3             10127     
Dependent_count                int64           10127     
Education_Level                <U15            10127     
Marital_Status                 <U10            10127     
Income_Category                <U16            10127     
Card_Category                  <U10            10127     
Months_on_book                 int64           10127     
Total_Relationship_Count       int64           10127     
Months_Inactive_12_mon         int64           10127     
Contacts_Count_12_mon          int64           10127     
Credit_Limit                   float64         10127     
Total_Revolving_Bal            int64           10127     
Avg_Open_To

**Nhận xét:** Có 10127 dòng dữ liệu, mỗi cột cũng có chừng ấy dòng --> không có missing values.

In [5]:
# Quan sát thông kê của các cột số
describe_all(data_np, include=True)

THỐNG KÊ CÁC CỘT SỐ

Cột: CLIENTNUM
------------------------------------------------------------
  Mean           : 739177606.33
  Median         : 717926358.00
  Std            :  36901961.36
  Min            : 708082083.00
  Max            : 828343083.00
  25%            : 713036770.50
  75%            : 773143533.00

Cột: Customer_Age
------------------------------------------------------------
  Mean           :        46.33
  Median         :        46.00
  Std            :         8.02
  Min            :        26.00
  Max            :        73.00
  25%            :        41.00
  75%            :        52.00

Cột: Dependent_count
------------------------------------------------------------
  Mean           :         2.35
  Median         :         2.00
  Std            :         1.30
  Min            :         0.00
  Max            :         5.00
  25%            :         1.00
  75%            :         3.00

Cột: Months_on_book
-----------------------------------------------

**Nhận xét & Insight ban đầu từ thống kê mô tả**

Dựa vào bảng số liệu trên, em rút ra 5 điểm đáng chú ý sau:

1.  **Hiện tượng Mất cân bằng dữ liệu (Imbalanced Data):**
    * Cột `Attrition_Flag` cho thấy **83.93%** là khách hàng hiện tại (Existing) và chỉ khoảng **16.07%** là khách hàng rời bỏ (Attrited).
    * *Insight:* Đây là vấn đề kinh điển trong bài toán Churn. Nếu model chỉ đoán toàn bộ là "Existing", độ chính xác vẫn đạt 84% nhưng model đó vô dụng. Chúng ta cần lưu ý điều này khi chọn metric đánh giá (nên dùng F1-score thay vì Accuracy).

2.  **Phân phối lệch của Hạn mức tín dụng (`Credit_Limit`):**
    * Mean (8,631) lớn hơn rất nhiều so với Median (4,549).
    * *Insight:* Dữ liệu bị lệch phải (Right-skewed). Điều này nghĩa là đa số khách hàng có hạn mức thấp, nhưng có một nhóm nhỏ "khách hàng VIP" có hạn mức cực cao kéo trung bình lên. Khi xử lý, có thể cần chuẩn hóa (Log transformation) để giảm bớt độ lệch này.

3.  **Hành vi sử dụng thẻ khá "thận trọng" (`Avg_Utilization_Ratio`):**
    * Trung bình tỷ lệ sử dụng là 27%, nhưng giá trị trung vị (Median) chỉ là 18%. Thậm chí 25% số người dùng (Q1) hầu như không dùng thẻ (0.02).
    * *Insight:* Khách hàng trong tập dữ liệu này có xu hướng ít nợ nần hoặc ít quẹt thẻ so với hạn mức được cấp.

4.  **Sự áp đảo của thẻ hạng phổ thông (`Card_Category`):**
    * **93.18%** khách hàng sử dụng thẻ hạng **Blue**. Các hạng Silver, Gold, Platinum chiếm tỷ lệ quá nhỏ.
    * *Insight:* Biến này có phương sai thấp (low variance). Việc phân loại dựa trên hạng thẻ có thể sẽ khó khăn vì mẫu cho các hạng thẻ cao cấp quá ít để model học được quy luật.

5.  **Dấu hiệu cần làm sạch dữ liệu :** (TODO)
    * Xuất hiện 2 cột lạ: `Naive_Bayes_Classifier_...`. Đây có thể là "rác" từ một quy trình xử lý trước đó của người tạo dataset.
    * *Chiến lược:* Hai cột này hoàn toàn vô nghĩa về mặt ngữ nghĩa kinh doanh và cần phải bị **loại bỏ** ngay trong bước Tiền xử lý (Preprocessing) để tránh gây nhiễu model.

## 2.2. Quan sát từng cột

### Ma trận tương quan